# Generate maps of metrics

This notebook shows how to generate maps of various metrics



In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import os
import geopandas as gpd
import pandas as pd
import numpy as np

In [3]:
from os import chdir
if "_changed_dir" not in locals():
    chdir("..")
    _changed_dir = True

In [4]:
from model_analysis import *

## Load base data

### Load geographic data


In [5]:
geo = gpd.read_file('./data/processed/geo/covid19model_zones.shp')  

### Load run manifests

In [6]:
run_manifest = read_manifest('run-manifest.csv')
analysis_set = get_analysis_set(run_manifest)

### select a run

In [7]:
run_data = analysis_set.iloc[-1,]

### Match country names between geo and model

In [43]:
geo_fra_regions = ["France", *[i for i in geo.name[geo.geo_level == "REG"].unique()]]
model_regions = run_data.model.data["ifr"].region
model_countries = run_data.model.data["ifr"].country

prob_match = {}
for zone in model_regions[model_regions != model_countries].unique():
    prob_match[zone] = icl_translate.most_probable_match(zone, geo_fra_regions)

# Print French 
print(geo_fra_regions)

# Hard code failures (due to complete name mismatch)
prob_match['Grand Est'] = 'ALSACE-CHAMPAGNE-ARDENNE-LORRAINE'
prob_match['Hauts-de-France'] = 'NORD-PAS-DE-CALAIS-PICARDIE'
prob_match['Nouvelle-Aquitaine'] = 'AQUITAINE-LIMOUSIN-POITOU-CHARENTES'
prob_match['Occitanie'] = 'LANGUEDOC-ROUSSILLON-MIDI-PYRENEES'

Match 1 of 1 : 'AUVERGNE-RHONE-ALPES' == 'Auvergne-Rhône-Alpes'  with a match score of 35/80.
	Match 1 of 1 : 'BOURGOGNE-FRANCHE-COMTE' == 'Bourgogne-Franche-Comté'  with a match score of 49/92.
	Match 1 of 1 : 'BRETAGNE' == 'Bretagne'  with a match score of 18/32.
	Match 1 of 1 : 'CENTRE-VAL DE LOIRE' == 'Centre-Val de Loire'  with a match score of 46/76.
	Match 1 of 1 : 'CORSE' == 'Corse'  with a match score of 12/20.
	Match 1 of 1 : 'France' == 'Grand Est'  with a match score of 9/36.
	Match 1 of 1 : 'France' == 'Hauts-de-France'  with a match score of 24/60.
	Match 1 of 1 : 'NORMANDIE' == 'Normandie'  with a match score of 20/36.
	Match 1 of 1 : 'GUYANE' == 'Nouvelle-Aquitaine'  with a match score of 4/72.
	Match 1 of 1 : 'France' == 'Occitanie'  with a match score of 4/36.
	Match 1 of 1 : 'PAYS DE LA LOIRE' == 'Pays de la Loire'  with a match score of 39/64.
	Match 1 of 1 : 'PROVENCE-ALPES-COTE D'AZUR' == 'Provence-Alpes-Côte d'Azur'  with a match score of 49/104.
	Match 1 of 1 : 

In [44]:
geo_countries = geo.name[geo.geo_level == "CNTRY"].unique()
for zone in model_countries.unique():
    prob_match[zone] = icl_translate.most_probable_match(zone, geo_countries)

Match 1 of 1 : 'France' == 'France'  with a match score of 24/24.
	Match 1 of 1 : 'Denmark' == 'Denmark'  with a match score of 28/28.
	Match 1 of 1 : 'Italy' == 'Italy'  with a match score of 20/20.
	Match 1 of 1 : 'Germany' == 'Germany'  with a match score of 28/28.
	Match 1 of 1 : 'Spain' == 'Spain'  with a match score of 20/20.
	Match 1 of 1 : 'United Kingdom' == 'United_Kingdom'  with a match score of 38/56.
	Match 1 of 1 : 'Norway' == 'Norway'  with a match score of 24/24.
	Match 1 of 1 : 'Belgium' == 'Belgium'  with a match score of 28/28.
	Match 1 of 1 : 'Austria' == 'Austria'  with a match score of 28/28.
	Match 1 of 1 : 'Sweden' == 'Sweden'  with a match score of 24/24.
	Match 1 of 1 : 'Switzerland' == 'Switzerland'  with a match score of 44/44.
	Match 1 of 1 : 'Greece' == 'Greece'  with a match score of 24/24.
	Match 1 of 1 : 'Portugal' == 'Portugal'  with a match score of 32/32.
	Match 1 of 1 : 'Netherlands' == 'Netherlands'  with a match score of 44/44.


In [46]:
for mode_zone in prob_match: 
    print(f"{mode_zone}: {prob_match[mode_zone]}")

Auvergne-Rhône-Alpes: AUVERGNE-RHONE-ALPES
Bourgogne-Franche-Comté: BOURGOGNE-FRANCHE-COMTE
Bretagne: BRETAGNE
Centre-Val de Loire: CENTRE-VAL DE LOIRE
Corse: CORSE
Grand Est: ALSACE-CHAMPAGNE-ARDENNE-LORRAINE
Hauts-de-France: NORD-PAS-DE-CALAIS-PICARDIE
Normandie: NORMANDIE
Nouvelle-Aquitaine: AQUITAINE-LIMOUSIN-POITOU-CHARENTES
Occitanie: LANGUEDOC-ROUSSILLON-MIDI-PYRENEES
Pays de la Loire: PAYS DE LA LOIRE
Provence-Alpes-Côte d'Azur: PROVENCE-ALPES-COTE D'AZUR
Île-de-France: ILE-DE-FRANCE
France-hopitaux: France
France-OC19: France
France-EHPAD: France
France: France
Denmark: Denmark
Italy: Italy
Germany: Germany
Spain: Spain
United_Kingdom: United Kingdom
Norway: Norway
Belgium: Belgium
Austria: Austria
Sweden: Sweden
Switzerland: Switzerland
Greece: Greece
Portugal: Portugal
Netherlands: Netherlands


## Define processing functions